In [9]:
import snowflake.connector
import pandas as pd

# Connect to Snowflake
conn = snowflake.connector.connect(
    user='tsethi',
    password='Tsethi@7654321',
    account='ml69805.us-east-1',
    warehouse='AI_MEDIUM',
    database='prod_tapintel',
    schema='public',
    role='public'
)
cur = conn.cursor()

table_name = 'TBL__ADWORDS_KEYWORDLESS_QUERY'

# Fetch column names and data types
cur.execute(f"""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_name = '{table_name}'
      AND table_schema = 'PUBLIC'
      AND table_catalog = 'PROD_TAPINTEL';
""")
columns = cur.fetchall()

# Prepare result list
results = []

for col, dtype in columns:
    stats = {'column': col, 'data_type': dtype}
    
    if dtype.lower() in ['number', 'int', 'float', 'decimal']:
        query = f"""
            SELECT 
                COUNT(*) AS total_rows,
                COUNT(DISTINCT {col}) AS unique_values,
                COUNT(*) - COUNT({col}) AS null_count,
                CASE 
                    WHEN COUNT(*) = 0 THEN 0 
                    ELSE ROUND((COUNT(*) - COUNT({col})) * 100.0 / COUNT(*), 2) 
                END AS null_percentage,
                ROUND(MIN({col}), 2) AS min_value,
                ROUND(MAX({col}), 2) AS max_value,
                ROUND(AVG({col}), 2) AS avg_value,
            FROM prod_tapintel.public.{table_name};
        """
    else:
        query = f"""
            SELECT 
                COUNT(*) AS total_rows,
                COUNT(DISTINCT {col}) AS unique_values,
                COUNT(*) - COUNT({col}) AS null_count,
                CASE 
                    WHEN COUNT(*) = 0 THEN 0 
                    ELSE ROUND((COUNT(*) - COUNT({col})) * 100.0 / COUNT(*), 2) 
                END AS null_percentage
            FROM prod_tapintel.public.{table_name};
        """
    
    cur.execute(query)
    row = cur.fetchone()
    
    # Combine stats into the dictionary
    if row:
        keys = [desc[0].lower() for desc in cur.description]
        stats.update(dict(zip(keys, row)))
    
    results.append(stats)

cur.close()
conn.close()

# Convert to DataFrame
df_stats = pd.DataFrame(results)
pd.options.display.float_format = '{:,.2f}'.format

# Optionally mark redundant columns
df_stats['is_redundant'] = df_stats.get('null_percentage', 0) == 100.0


                         column      data_type  total_rows  unique_values  \
0                  QUERY_STATUS           TEXT           0              0   
1                           URL           TEXT           0              0   
2                         QUERY           TEXT           0              0   
3   LAST_UPDATE_MYSQL_TIMESTAMP  TIMESTAMP_NTZ           0              0   
4                    ADGROUP_ID         NUMBER           0              0   
5                          NAME           TEXT           0              0   
6                       WEBPAGE           TEXT           0              0   
7                        DOMAIN           TEXT           0              0   
8           KEYWORDLESSQUERY_ID         NUMBER           0              0   
9                      HEADLINE           TEXT           0              0   
10                      NETWORK           TEXT           0              0   
11           LANDING_PAGE_TITLE           TEXT           0              0   

In [10]:
df_stats.head(10000)

,column,data_type,total_rows,unique_values,null_count,null_percentage,min_value,max_value,avg_value,is_redundant
0,QUERY_STATUS,TEXT,0,0,0,0.00,NaN,NaN,NaN,False
1,URL,TEXT,0,0,0,0.00,NaN,NaN,NaN,False
2,QUERY,TEXT,0,0,0,0.00,NaN,NaN,NaN,False
3,LAST_UPDATE_MYSQL_TIMESTAMP,TIMESTAMP_NTZ,0,0,0,0.00,NaN,NaN,NaN,False
4,ADGROUP_ID,NUMBER,0,0,0,0.00,NaN,NaN,NaN,False
5,NAME,TEXT,0,0,0,0.00,NaN,NaN,NaN,False
6,WEBPAGE,TEXT,0,0,0,0.00,NaN,NaN,NaN,False
7,DOMAIN,TEXT,0,0,0,0.00,NaN,NaN,NaN,False
8,KEYWORDLESSQUERY_ID,NUMBER,0,0,0,0.00,NaN,NaN,NaN,False
9,HEADLINE,TEXT,0,0,0,0.00,NaN,NaN,NaN,False
